# Data Processing

## Imports

In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

## Data Exploration

In [22]:
df = pd.read_csv('../data/sp500_dataset.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Drop unneeded columns for efficiency, these columns will be preesent in the master dataset for later reference if needed.
df = df.drop(columns=['Open', 'High', 'Low', 'Volume'])
df.head()

C:\Users\dpate\AppData\Local\Temp\ipykernel_1752\3578041768.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Date'] = pd.to_datetime(df['Date'])


,Date,Close,Dividends,Stock Splits,Ticker,Adj Close
0,2020-08-10 00:00:00-04:00,109.776482,0.0,0.0,AAPL,NaN
1,2020-08-11 00:00:00-04:00,106.511757,0.0,0.0,AAPL,NaN
2,2020-08-12 00:00:00-04:00,110.051590,0.0,0.0,AAPL,NaN
3,2020-08-13 00:00:00-04:00,111.999237,0.0,0.0,AAPL,NaN
4,2020-08-14 00:00:00-04:00,111.899429,0.0,0.0,AAPL,NaN


Check how much data each company has. We want to only keep the ones with enough meaningful data.

In [23]:
sub_df = df.groupby('Ticker').size().sort_values(ascending=False)
print(sub_df.unique())
sub_df[-12:]

[1256 1241 1240 1230 1220 1170  892  663  568  463  344  343  219]


Ticker
HES     1241
ANSS    1240
JNPR    1230
PLTR    1220
ABNB    1170
CEG      892
GEHC     663
KVUE     568
VLTO     463
SOLV     344
GEV      343
AMTM     219
dtype: int64

Drop the companies with more than approx 30 days of missing data.

In [24]:
drop_list = ['ABNB', 'CEG', 'GEHC', 'KVUE', 'VLTO', 'SOLV', 'GEV', 'AMTM']
for c in drop_list:
    df = df[df['Ticker'] != c]

# Verify the companies were dropped
sub_df = df.groupby('Ticker').size().sort_values(ascending=False)
print(sub_df.unique())

[1256 1241 1240 1230 1220]


Use forward-fill to fill in the remaining missing rows from the companies that remain

In [33]:
df = df.sort_index()
df = df.ffill(axis=0)

# Verify
sub_df = df.groupby('Ticker').size().sort_values(ascending=False)
print(sub_df.unique())

print(sub_df)


[1256 1241 1240 1230 1220]
Ticker
ZTS     1256
A       1256
AAPL    1256
ABBV    1256
ABT     1256
        ... 
AES     1256
HES     1241
ANSS    1240
JNPR    1230
PLTR    1220
Length: 493, dtype: int64


Next, we want to account for dividends and stock splits in our dataset. This will help populate the 'Adj Close' feature column. 

In [ ]:
# Sort the DF by Ticker and Date
df.sort_values(by=['Ticker', 'Date'], inplace=True)

calc_df = df.copy()

# The factor to change based on splits depends on the number of splits
# For example, if a company splits 2:1, then the factor is 1/2
splits = calc_df['Stock Splits'].replace(0, 1).fillna(1)
split_step = 1 / splits 

# Adjust for dividents as well
# Adjust the price based on factor = 1 - (Dividend / Close)
div_step = 1 - (calc_df['Dividends'] / calc_df['Close']).where((calc_df['Dividends'] > 0) & (calc_df['Close'] > 0), 0)

combined_step = (split_step * div_step).astype('float64')

# Sort it in reverse order for the next step: applying the factor
calc_df.sort_values(by=['Ticker', 'Date'], ascending=[True, False], inplace=True) 

# Calculate the in reverse order, with the most recent date having a factor of 1. representing no change
factor = (combined_step.groupby(calc_df['Ticker']).transform(lambda s: s.shift(1).fillna(1.0).cumprod()))

calc_df['Factor'] = factor

# Bring back chronological order
calc_df.sort_values(['Ticker', 'Date'])

# Calculate adjusted close
calc_df['Adj. Close'] = calc_df['Close'] * calc_df['Factor']

In [21]:
# Verify the Adj. Close calculations worked as expected
test_df = calc_df[calc_df['Ticker'] == 'AAPL']

leaks = calc_df.sort_values(['Ticker','Date']).groupby('Ticker').tail(1)['Factor'].unique()
print(leaks[:5], '...')  # expect array([1.])


[0.96631299 0.24339081 0.79831187 0.91041545 0.94844298] ...
